In [1]:
from cmp.pycompiler import *
from TreeDef import *

In [ ]:

# G = Grammar()

# # No terminales
# E = G.NonTerminal('E', True)
# A,T,F = G.NonTerminals('A T F')

# # Terminales
# equal, plus, times, num, lpar, rpar = G.Terminals('= + * float ( )')

# # Reglas
# E %= A, lambda h,s : s[1]

# A %=  A + plus + T, lambda h,s: PlusNode(s[1],s[3])
# A %= T, lambda h,s : s[1]

# T %= T + times + F, lambda h,s: StarNode(s[1],s[3])
# T %= F, lambda h,s : s[1]

# F %= num, lambda h,s : ConstantNumberNode(s[1])
# F %= lpar + A + rpar, lambda h,s : s[2]


In [3]:

G1 = Grammar()

# No terminales
E = G1.NonTerminal('E', True)
T,F = G1.NonTerminals('T F')

# Terminales
equal, plus, times, num, lpar, rpar = G1.Terminals('= + * float ( )')

# Reglas
# E %= A, lambda h,s : s[1]

E %=  E + plus + T, lambda h,s: PlusNode(s[1],s[3])
E %= T, lambda h,s : s[1]

T %= T + times + F, lambda h,s: StarNode(s[1],s[3])
T %= F, lambda h,s : s[1]

F %= num, lambda h,s : ConstantNumberNode(s[1])
F %= lpar + E + rpar, lambda h,s : s[2]


In [4]:
from cmp.pycompiler import Item

item = Item(E.productions[0], 0, lookaheads=[G1.EOF, plus])
print('item:', item)

item: E -> .E+T, {'$', '+'}


In [5]:

from cmp.utils import ContainerSet
from cmath import e


def compute_local_first(firsts, alpha):
    first_alpha = ContainerSet()
    
    try:
        alpha_is_epsilon = alpha.IsEpsilon
    except:
        alpha_is_epsilon = False
    
    ###################################################
    # alpha == epsilon ? First(alpha) = { epsilon }
    ###################################################
    if(alpha_is_epsilon):
        first_alpha.set_epsilon(True)
        return first_alpha
    ###################################################
    
    ###################################################
    # alpha = X1 ... XN
    # First(Xi) subconjunto First(alpha)
    # epsilon pertenece a First(X1)...First(Xi) ? First(Xi+1) subconjunto de First(X) y First(alpha)
    # epsilon pertenece a First(X1)...First(XN) ? epsilon pertence a First(X) y al First(alpha)
    ###################################################
    for symbol in alpha:
        first_symbol = firsts[symbol]
        first_alpha.update(first_symbol)
        if(not first_symbol.contains_epsilon):
            break
            
    ###################################################
    
    # First(alpha)
    return first_alpha

def compute_firsts(G):
    firsts = {}
    change = True
    
    # init First(Vt)
    # Each terminal has a First set with only itself
    for terminal in G.terminals:
        firsts[terminal] = ContainerSet(terminal)
    
    # init First(Vn)
    # Each non-terminal is initialized with an empty set
    for nonterminal in G.nonTerminals:
        firsts[nonterminal] = ContainerSet()
    
    while change:
        change = False
        
        # P: X -> alpha
        for production in G.Productions:
            X = production.Left
            alpha = production.Right
            
            # get current First(X)
            first_X = firsts[X]
                
            # init First(alpha)
            try:
                first_alpha = firsts[alpha]
            except KeyError:
                first_alpha = firsts[alpha] = ContainerSet()
            
            # CurrentFirst(alpha)???
            local_first = compute_local_first(firsts, alpha)
            # update First(X) and First(alpha) from CurrentFirst(alpha)
            change |= first_alpha.hard_update(local_first)
            change |= first_X.hard_update(local_first)
                    
    # First(Vt) + First(Vt) + First(RightSides)
    return firsts


In [6]:
firsts = compute_firsts(G1)
firsts[G1.EOF] = ContainerSet(G1.EOF)

In [7]:
firsts

{'=': {'='}-False,
 '+': {'+'}-False,
 '*': {'*'}-False,
 'float': {'float'}-False,
 '(': {'('}-False,
 ')': {')'}-False,
 'E': {'(', 'float'}-False,
 'T': {'(', 'float'}-False,
 'F': {'(', 'float'}-False,
 E + T: {'(', 'float'}-False,
 T: {'(', 'float'}-False,
 T * F: {'(', 'float'}-False,
 F: {'(', 'float'}-False,
 float: {'float'}-False,
 ( E ): {'('}-False,
 '$': {'$'}-False}